In [1]:
import string # for lowercase letter list

In [2]:
with open('/usr/share/dict/words') as o:
    DICTIONARY = set([s.strip() for s in o.readlines() if not s[0].isupper()])

In [3]:
class Mutator:
    def __init__(self, dictionary):
        self.letters =  string.ascii_lowercase
        self.dictionary = dictionary
        
    def __call__(self, word):
        """List all the valid point mutations that can be made to a word"""
        words = []

        # Insertions
        for position in range(len(word)+1):
            for letter in self.letters:
                new_word = word[:position] + letter + word[position:]
                if new_word in self.dictionary:
                    words.append(new_word)

        # Removals
        for position in range(len(word)):
            new_word = word[:position] + word[position+1:]
            if new_word in self.dictionary:
                words.append(new_word)

        # Substitutions
        for position in range(len(word)):
            for letter in (l for l in self.letters if not l == word[position]):
                new_word = word[:position] + letter + word[position+1:]
                if new_word in self.dictionary:
                    words.append(new_word)

        # Keep only valid words
        return [word for word in words if word in self.dictionary]     

In [5]:
class WordPath:
    def __init__(self, word, path = None):
        self._head = word
        self._tail = path
        
    def step(self, word):
        return WordPath(word, self)
        
    @property    
    def head(self):
        return self._head
        
    def __repr__(self):
        words = [self._head]
        current = self
        while current._tail:
            current = current._tail
            words = [current._head] + words
        return ", ".join(words)

In [6]:
class WordTree:
    def __init__(self, word:str):
        self.paths = [WordPath(word)]
        self.words = set()
        
        self.mutator = Mutator(DICTIONARY)
        
    def step(self, n=1):
        new_paths = []
        for path in self.paths:
            next_words = self.mutator(path.head)
            for next_word in next_words:
                if next_word in self.words:
                    continue
                new_paths.append(path.step(next_word))
                self.words.add(next_word)
        if not new_paths:
            raise RuntimeError("No path could be found")
        self.paths = new_paths
        
    def __repr__(self):
        return "<WordTree([{}])>".format(", ".join(str(path) for path in self.paths))

In [7]:
def find_path(start, end):
    wordTree = WordTree(start)
    while True:
        wordTree.step()
        for path in wordTree.paths:
            if path.head == end:
                return path

In [12]:
find_path("cat", "basket")

cat, cast, caste, caster, baster, basker, basket